## Rotation matrices for DM projection to healpix map.

In [ ]:
from shared.preface import *
from shared.shared_functions import *

init_dis = 8.5
orig_xyz = np.array([init_dis, 0, 0.])
ic(orig_xyz)
xE = np.cos(np.deg2rad(Pi))*np.sin(np.deg2rad(Pi))*init_dis
yE = np.sin(np.deg2rad(Pi))*np.sin(np.deg2rad(Pi))*init_dis
zE = np.cos(np.deg2rad(Pi))*init_dis
init_xyz = np.array([xE, yE, zE])
# ic(init_xyz)

# Rotations happen in clockwise direction, when viewed from the positive top of 
# the axis around which rotation happends. 
# note: See GoodNotes drawing, for orientation of axes the functions assume.

# Earth was on x-axis originally. Applying matrix with positive angles will 
# result in going to the new earth frame, where now the original position is 
# seen from earths frame.
# This matrix must be used, to get all the DM in this new frame, such that we 
# can project it on the healpix map#!
order = 'zyx'
z_rot = Pi
y_rot = -Pi
x_rot = 0
rotation_mat_for_map = rotation_matrix(z_rot, y_rot, x_rot, order)
# ic(rotation_mat)
# angles = rotation_angles(rotation_mat, order)
# print(angles)
rot_xyz = np.matmul(rotation_mat_for_map, orig_xyz)
ic(rot_xyz)

# Rotation matrix with negative angles must be used, to get earths slightly 
# shifted position (needed for the cell edge issue) in the original frame, 
# where all the simulation happens#!
rotation_mat_for_sim = rotation_matrix(-z_rot, -y_rot, -x_rot, order)


### TEST ###

# Take a DM particle in the original frame with coordinates (x,y,z)_DM and 
# earth position in the same frame with (x,y,z)_E, where Earth is not 
# positioned on the x-axis. The only thing that we have to do, to "rotate" the 
# DM particle into the frame, where Earth is again on the x-axis connecting it 
# to the halo center, is to multiply (x,y,z)_DM by the rotation matrix, which 
# brought earth to be at the location in the original frame in the first place.

# note: We can get the galactic latitude and galactic longitude in the frame, where Earth lies on the x-axis (see drawings) connecting it to the halo center.

# Step 1: Original positions.
DM_test = np.array([-5, 1, 5])  # DM particle in original frame

# Offset Earth position from x-axis (because of the cell edge issue).
rot_mat = rotation_matrix(0, 0, 20, order)

# DM particle in earths frame.
DM_earth = np.matmul(rot_mat, DM_test)
ic('************')
ic(DM_test)
ic(DM_earth)

## Visual of all colorcet maps.

## Old stuff: to be analyzed and documented.

In [13]:
def nu_in_which_cell(x_i, cell_coords, cell_gens, init_GRID_S):

    # print(x_i.shape)  # (3,)
    # print(cell_coords.shape)  # (cells, 1, 3)
    # print(cell_gens.shape)  # (cells,)

    # Center neutrino coords. on each cell center (whole grid).
    x_i = np.repeat(np.expand_dims(x_i, axis=(0,1)), len(cell_coords), axis=0)
    x_i -= cell_coords

    # All cell lengths. Limit for the largest cell is GRID_S/2, not just 
    # GRID_S, therefore the cell_gen+1 !
    cell_lens = np.expand_dims(init_GRID_S/(2**(cell_gens+1)), axis=1)
    # print(cell_lens.shape)  # (cells, 1)
    # print(x_i[...,0].shape)  # (cells, 1)

    # Find index of cell in which neutrino is enclosed.
    in_cell = np.asarray(
        (np.abs(x_i[...,0]) < cell_lens) & 
        (np.abs(x_i[...,1]) < cell_lens) & 
        (np.abs(x_i[...,2]) < cell_lens)
    )
    cell_idx = np.argwhere(in_cell==True).flatten()[0]  # np.int64

    return cell_idx


# Load files.
out_dir = 'L025N752/DMONLY/SigmaConstant00'
temp_dir = f'{out_dir}/temp_data_TEST'
cell_coords = np.load(f'{temp_dir}/fin_grid_origID28_snap_0014.npy')
cell_gens = np.load(f'{temp_dir}/cell_gen_origID28_snap_0014.npy')
init_GRID_S = np.load(f'{out_dir}/snaps_GRID_L.npy')[2]
print('Cells for this snap:', len(cell_coords))

with open(f'{out_dir}/sim_parameters.yaml', 'r') as file:
    sim_setup = yaml.safe_load(file)

init_dis = sim_setup['initial_haloGC_distance']
xE = np.cos(np.deg2rad(Pi))*np.sin(np.deg2rad(Pi))*init_dis
yE = np.sin(np.deg2rad(Pi))*np.sin(np.deg2rad(Pi))*init_dis
zE = np.cos(np.deg2rad(Pi))*init_dis
x_i = np.array([xE, yE, zE])*kpc

cell_idx = nu_in_which_cell(x_i, cell_coords, cell_gens, init_GRID_S)

Cells for this snap: 50
(3,)
(50, 1, 3)
(50,)
(50, 1)
(50, 1)
<class 'numpy.int64'>


In [6]:
from itertools import zip_longest

x1 = np.arange(3)+6
x2 = np.arange(5)+6

y1 = np.arange(3)+25
y2 = np.arange(5)+25

lx1d = [x1,x2]
ly1d = [y1,y2]

arrx2d = np.array(list(zip_longest(*lx1d, fillvalue=np.nan))).T
arry2d = np.array(list(zip_longest(*ly1d, fillvalue=np.nan))).T

print(arrx2d)
print(arrx2d.shape)
print(arry2d)
print(arry2d.shape)

fin = np.stack((arrx2d, arry2d), axis=2)
print(fin)

[[ 6.  7.  8. nan nan]
 [ 6.  7.  8.  9. 10.]]
(2, 5)
[[25. 26. 27. nan nan]
 [25. 26. 27. 28. 29.]]
(2, 5)
[[[ 6. 25.]
  [ 7. 26.]
  [ 8. 27.]
  [nan nan]
  [nan nan]]

 [[ 6. 25.]
  [ 7. 26.]
  [ 8. 27.]
  [ 9. 28.]
  [10. 29.]]]


In [16]:
from itertools import product, combinations

arra = [[1,2],[3,4],[5,6]]
arrb = [4,5,6]
arrc = np.array(list(combinations(arra, 2)))
print(arrc)
print(np.product(arrc, axis=1))

[[[1 2]
  [3 4]]

 [[1 2]
  [5 6]]

 [[3 4]
  [5 6]]]
[[ 3  8]
 [ 5 12]
 [15 24]]


In [4]:
count_arr = np.arange(4)+2
print(count_arr)
count_arr[0] = 0.
count_arr[1] = 0.
count_arr[2] = 0.
# count_arr[3] = 0.
print(count_arr)

if np.all(count_arr==0):
    DM_axis0 = np.full(shape=(len(count_arr),1,3), fill_value=np.nan)
else:
    x_arr = np.arange(np.sum(count_arr))
    breaks = np.cumsum(count_arr[:-1])
    x_split = np.split(x_arr, breaks)
    # print(x_split)
    DM_axis0 = np.array(list(zip_longest(*x_split, fillvalue=np.nan))).T


print(DM_axis0)

[2 3 4 5]
[0 0 0 5]
[[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [ 0.  1.  2.  3.  4.]]


In [9]:
DM_count_mpoles = np.array([2])

x_arr = np.arange(np.sum(DM_count_mpoles))
breaks = np.cumsum(DM_count_mpoles[:-1])
x_split = np.split(x_arr, breaks)
print(x_split)
DM_axis0 = np.array(list(zip_longest(*x_split, fillvalue=np.nan))).T
DM_axis1 = np.array(list(zip_longest(*x_split, fillvalue=np.nan))).T
DM_axis2 = np.array(list(zip_longest(*x_split, fillvalue=np.nan))).T
print(DM_axis0)
DM_mpoles = np.stack((DM_axis0, DM_axis1, DM_axis2), axis=2)
print(DM_mpoles)

[array([0, 1])]
[[0 1]]
[[[0 0 0]
  [1 1 1]]]


shapes/nums: 267 (267,) 1 (3,) (267, 3) (267,) 394.0 (1, 457240, 3) (267,) (457239, 2)

In [7]:
from shared.preface import *
import shared.functions as fct

PRE = PRE(
    sim='L025N752', 
    z0_snap=36, z4_snap=13, DM_lim=1000,
    sim_dir=SIM_ROOT, sim_ver=SIM_TYPE,
    phis=20, thetas=20, vels=200,
    pre_CPUs=10, sim_CPUs=128
)


# Make temporary folder to store files, s.t. parallel runs don't clash.
rand_code = ''.join(
    random.choices(string.ascii_uppercase + string.digits, k=4)
)
TEMP_DIR = f'{PRE.OUT_DIR}/temp_data_{rand_code}'
os.makedirs(TEMP_DIR)

Testing = False
if Testing:
    mass_gauge = 12.3
    mass_range = 0.3
    size = 1
else:
    mass_gauge = 12.0
    mass_range = 0.6
    size = 10

hname = f'1e+{mass_gauge}_pm{mass_range}Msun'
fct.halo_batch_indices(
    PRE.Z0_STR, mass_gauge, mass_range, 'halos', size, 
    hname, PRE.SIM_DIR, TEMP_DIR
)
halo_batch_IDs = np.load(f'{TEMP_DIR}/halo_batch_{hname}_indices.npy')
halo_batch_params = np.load(f'{TEMP_DIR}/halo_batch_{hname}_params.npy')
halo_num = len(halo_batch_params)

print('********Number density band********')
print('Halo batch params (Rvir,Mvir,cNFW):')
print(halo_batch_params)
print('***********************************')

# Remove temporary folder with all individual neutrino files.
shutil.rmtree(TEMP_DIR)

********************* Initialization *********************
# Initial conditions for neutrinos:
PHIs = 20, THETAs=20, Vs=200
Total neutrinos: 80000
# Simulation parameters:
Simulation box: L025N752
Snapshot from 0036 (z=0) to 0013 (z=4)
Pre/Sim CPUs 10/128
DM limit for cells: 1000
# File management:
Box files directory: 
 /projects/0/einf180/Tango_sims/L025N752/DMONLY/SigmaConstant00
Output directory: 
 /gpfs/home4/zimmer/neutrino_clustering_V2/L025N752/DMONLY/SigmaConstant00
**********************************************************
********Number density band********
Halo batch params (Rvir,Mvir,cNFW):
[[253.5523526   12.24058042   6.9286026 ]
 [158.62979351  11.62953177   8.03988676]
 [137.07912107  11.43929142   9.41292697]
 [141.82849159  11.48366794   2.6088865 ]
 [306.35586267  12.48705608  11.48652354]
 [190.92998087  11.87099988   7.03921462]
 [240.52254202  12.17184485   7.39127504]
 [169.32112452  11.71451092   8.23421103]
 [167.71139596  11.70206521   7.62954256]
 [168.53268

In [1]:
from shared.preface import *
import shared.functions as fct

# Initialize parameters and files.
PRE = PRE(
    sim='L012N376', 
    z0_snap=36, z4_snap=13, DM_lim=1000,
    sim_dir=SIM_ROOT, sim_ver=SIM_TYPE,
    phis=10, thetas=10, vels=100,
    pre_CPUs=6, sim_CPUs=6
)

Testing=False
if Testing:
    mass_gauge = 12.3
    mass_range = 0.3
    size = 1
else:
    mass_gauge = 12.0
    mass_range = 0.6
    size = 10

hname = f'1e+{mass_gauge}_pm{mass_range}Msun'
fct.halo_batch_indices(
    PRE.Z0_STR, mass_gauge, mass_range, 'halos', size, 
    hname, PRE.SIM_DIR, PRE.OUT_DIR
)
halo_batch_IDs = np.load(f'{PRE.OUT_DIR}/halo_batch_{hname}_indices.npy')
halo_batch_params = np.load(f'{PRE.OUT_DIR}/halo_batch_{hname}_params.npy')
halo_num = len(halo_batch_params)

print('********Number density band********')
print('Halo batch params (Rvir,Mvir,cNFW):')
print(halo_batch_params)
print('***********************************')

********************* Initialization *********************
# Initial conditions for neutrinos:
PHIs = 10, THETAs=10, Vs=100
Total neutrinos: 10000
# Simulation parameters:
Simulation box: L012N376
Snapshot from 0036 (z=0) to 0013 (z=4)
Pre/Sim CPUs 6/6
DM limit for cells: 1000
# File management:
Box files directory: 
 /home/fabian/ownCloud/snellius/L012N376/DMONLY/SigmaConstant00
Output directory: 
 /home/fabian/my_github_projects/neutrino_clustering_V2/L012N376/DMONLY/SigmaConstant00
**********************************************************
16
********Number density band********
Halo batch params (Rvir,Mvir,cNFW):
[[195.00028262  11.89848321  13.2646342 ]
 [171.6865726   11.73258648   5.56997953]
 [139.87064301  11.46555721   7.37974116]
 [151.26621853  11.56760334   7.82480041]
 [298.72978142  12.45421305  10.25293176]
 [140.09726066  11.46766642  10.07694324]
 [153.19481535  11.58410969  10.62058499]
 [176.89228757  11.77150419   6.78838463]
 [173.01242844  11.7426094    5.41372435

# Tests for CubeSpace components.

# Tests for functions in shared/functions.py

### Optimal momentum spacing.

In [ ]:
from shared.preface import *
import shared.functions as fct

Vs = 200
LOWER = 0.01*T_CNB
UPPER = 10*T_CNB

fig, ax = plt.subplots(1,1)

v_tot_min = 1/np.sqrt(NU_MASSES[-1]**2/LOWER**2 + 1) / (kpc/s)
v_tot_max = 1/np.sqrt(NU_MASSES[0]**2/UPPER**2 + 1) / (kpc/s)
v_lines = np.geomspace(v_tot_min, v_tot_max, 5)

v_tot_geo = np.geomspace(v_tot_min, v_tot_max, Vs)
ax.scatter(v_tot_geo, v_tot_geo+(1.1*v_tot_geo), s=2)

for v0, v1 in zip(v_lines[:-1], v_lines[1:]):
    v_segment = np.geomspace(v0, v1, int(Vs/4))
    ax.scatter(v_segment, v_segment, s=2)
    ax.axvline(v0, ls=':', c='r')

plt.xscale('log')
plt.yscale('log')
plt.show()

In [ ]:
from shared.preface import *
import shared.functions as fct


fig, ax = plt.subplots(1,1)

momenta = np.geomspace(0.01*T_CNB, 10*T_CNB, 100)
v_kpc = 1/np.sqrt(NU_MASS**2/momenta**2 + 1) / (kpc/s)  # rel. formula
ax.scatter(v_kpc, v_kpc, s=2)

# plt.xscale('log')
# plt.yscale('log')
plt.show()

### Conversions between momentum to velocity.

In [3]:
from shared.preface import *
import shared.functions as fct

Vs = 100
LOWER = 0.01*T_CNB
UPPER = 400.*T_CNB

# Momentum range.
MOMENTA = np.geomspace(LOWER, UPPER, Vs)

# Without Lorentz factor.
v_mins = np.zeros(len(NU_MASSES))
v_maxs = np.zeros(len(NU_MASSES))
for i, m_nu in enumerate(NU_MASSES):
    v_km = MOMENTA / m_nu / (m/s)
    v_min, v_max = v_km[0], v_km[-1]
    # print(f'(m/s) -> v_min = {v_min:.2f}, v_max = {v_max:.2f}, %c = {v_max/const.c.value*100:.2f} : for {m_nu} eV neutrino')

    v_mins[i] = v_min
    v_maxs[i] = v_max

# With Lorentz factor.
v_mins = np.zeros(len(NU_MASSES))
v_maxs = np.zeros(len(NU_MASSES))
for m_nu in NU_MASSES:
    v_km = 1/np.sqrt(m_nu**2/MOMENTA**2 + 1) / (m/s)
    v_min, v_max = v_km[0], v_km[-1]
    print(f'(m/s) -> v_min = {v_min:.2f}, v_max = {v_max:.2f}, %c = {v_max/const.c.value*100:.2f} : for {m_nu} eV neutrino')

#! Since the sim is using 0.3 eV mass, the max. velocity present in the sim is 
#! ~20% of c, not the ~98% of the 0.01 eV neutrino (see output of cell).

# Back to momentum to check formulas & functions.

(m/s) -> v_min = 50256.48, v_max = 296513342.86, %c = 98.91 : for 0.01 eV neutrino
(m/s) -> v_min = 10051.30, v_max = 240334142.09, %c = 80.17 : for 0.05 eV neutrino
(m/s) -> v_min = 5025.65, v_max = 166963809.94, %c = 55.69 : for 0.1 eV neutrino
(m/s) -> v_min = 1675.22, v_max = 65394995.35, %c = 21.81 : for 0.3 eV neutrino


### Coordinates of Andromeda (AG) and the Virgo Cluster (VC).

In [ ]:
# Cross-check with values in Mertsch et al. (2020).
# -> x and y coords. are switched, since our setup is different,
# and my x (their y) coords. differ by 8.5 kpc due to placement of sun
# on our x-axis.
coords_VC = fct.halo_pos(GLAT_VC, GLON_VC, DIST_VC/kpc)
print(coords_VC, 'in kpc')
coords_AG = fct.halo_pos(GLAT_AG, GLON_AG, DIST_AG/kpc)
print(coords_AG, 'in kpc')

### Values in Table 1 of Mertsch et al. (2020).

In [ ]:
Rvir_Tab1 = fct.R_vir_fct(0, Mvir_MW)
print(Rvir_Tab1/kpc)
c_vir_Tab1 = fct.c_vir(0, Mvir_MW, Rvir_MW, Rs_MW)
Rs_Tab1 = Rvir_Tab1 / c_vir_Tab1
print(Rs_Tab1/kpc)

### Critical density of universe.

In [ ]:
rho_crit_today = fct.rho_crit(0)
print(f'{rho_crit_today*(Msun/kpc**3)/(kg/m**3):.2e} kg/m^3') 

### Time Variable s(z) and comparison to age of universe.

In [ ]:
# In s_of_z function we use
H0_mod = H0/ (1/s)
print(H0_mod)

test_z = 1
s_val = fct.s_of_z(test_z)
print(f'Value of time variable s in seconds at redhshift {test_z}:','\n', s_val)
print(
    'Age of universe comparison: \n', 
    f'"Observed/measured": {t0/s:.2e}, i.e. {t0/Gyr:.2f} Gyr \n', 
    f'What we use in s_of_z function: {1/H0/s:.2e}, i.e. {1/H0/Gyr:.2f} Gyr'
)

### Integrals for cosmic time.

In [ ]:
def t_integrand_a(a):

    # We need value of H0 in units of 1/s.
    H0_val = H0/(1/s)

    a_dot = np.sqrt(Omega_M/a**3 + Omega_L)*H0_val*a
    t_int = 1./a_dot

    return t_int

t, err = quad(t_integrand_a, 0, 1)
t_uni, err_uni = t, err
print(t_uni*s/Gyr, err_uni*s/Gyr)

In [ ]:
def t_integrand_z(z):

    # We need value of H0 in units of 1/s.
    H0_val = H0/(1/s)

    a_dot = np.sqrt(Omega_M*(1.+z)**3 + Omega_L)*H0_val*(1.+z)
    t_int = 1./a_dot

    return t_int

t, err = quad(t_integrand_z, 0, np.inf)
t_uni, err_uni = t, err
print(t_uni*s/Gyr, err_uni*s/Gyr)

### Fermi-Dirac distribution.

In [ ]:
p_test_range = np.linspace(0.01, 10)*T_CNB
FD_range = fct.Fermi_Dirac(p_test_range)

plt.loglog(p_test_range/T_CNB, FD_range)
plt.show()

### Redshift array for integration steps.

In [ ]:
# Linear spacing.
# late_steps = 200
# early_steps = 100
# Z_START, Z_STOP, Z_AMOUNT = 0., 4., late_steps+early_steps
# z_late = np.linspace(0,1,late_steps)
# z_early = np.linspace(1.01,4,early_steps)
# ZEDS = np.concatenate((z_late, z_early))

# Logarithmic spacing.
Z_AMOUNT = 50
z_shift = 1e-1
ZEDS = np.geomspace(z_shift, 4.+z_shift, Z_AMOUNT) - z_shift

plt.scatter(ZEDS, ZEDS, s=1)
plt.scatter(ZEDS_SNAPSHOTS, ZEDS_SNAPSHOTS+0.1, s=1)
plt.show()
print(ZEDS[0:10], ZEDS[-1])

### NFW density profile fct.

In [ ]:
r_range = np.geomspace(1e-3, 100, 100)*kpc
NFW_vals = fct.NFW_profile(r_range, rho0_MW, Rs_MW)
plt.loglog(r_range/kpc, NFW_vals/(GeV/cm**3))

### Typical grav. potential gradient vectors in spher. symmetric simulation.

In [ ]:
z = 0

# For position of Sun/Earth.
x_i = X_SUN*kpc
grad_MW = fct.dPsi_dxi_NFW(x_i, z, rho0_MW, Mvir_MW, Rvir_MW, Rs_MW, 'MW')
grad_MW /= (kpc/s**2)
print('Position of Sun/Earth:')
print(grad_MW)
print(np.sqrt(np.sum(grad_MW**2)), '\n')

# For a position closer to the center of the halo.
x_i = np.array([0.01, 0, 0])*kpc
grad_MW = fct.dPsi_dxi_NFW(x_i, z, rho0_MW, Mvir_MW, Rvir_MW, Rs_MW, 'MW')
grad_MW /= (kpc/s**2)
print('Position close to center:')
print(grad_MW)
print(np.sqrt(np.sum(grad_MW**2)), '\n')

# For a position further away to the center of the halo.
x_i = np.array([333., 0, 0])*kpc
grad_MW = fct.dPsi_dxi_NFW(x_i, z, rho0_MW, Mvir_MW, Rvir_MW, Rs_MW, 'MW')
grad_MW /= (kpc/s**2)
print('Position at virial radius:')
print(grad_MW)
print(np.sqrt(np.sum(grad_MW**2)))